In [2]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.metrics.pairwise import linear_kernel
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_extraction.text import TfidfVectorizer
# from collections import Counter
# import mysql.connector
# import warnings
# warnings.filterwarnings("ignore")



# # join and concatenate inputted columns
# def create_soup(x, features):
#     soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
#     return ' '.join(soup_parts)

# # count number of likeable elements for film
# def count_likeable(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if col is not None)
#     cast = len(row['cast'].split(','))
#     return atts+cast

# # get user loved films from db
# def get_loved_films(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     rows = mycursor.fetchall()

#     tconst_list = [tconst[0] for tconst in rows]

#     mycursor.close()
#     mydb.close()

#     loved_films_df = data[data['tconst'].isin(tconst_list)]

#     return loved_films_df

# # get user liked attributes from db
# def get_liked_attributes(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     attribute_fetch = mycursor.fetchall()

#     tconst_list = [row[1] for row in attribute_fetch]
#     attribute_bin = [row[1:] for row in attribute_fetch]

#     mycursor.close()
#     mydb.close()

#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     liked_attributes_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst, att_bin in zip(tconst_list, attribute_bin):
#         row_values = [tconst]  # Initialize row values with tconst
#         for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
#             film_att = data[data['tconst'] == tconst]

#             if liked == 1:
#                 row_values.append(film_att[attribute].values[0])
#             else:
#                 row_values.append(None) 
#         liked_attributes_df.loc[x] = row_values
#         x+=1

#     return liked_attributes_df

# # get user liked cast from db
# def get_liked_cast(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     cast_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     cast = [row[1:] for row in cast_fetch]
#     liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

#     return liked_cast_df

# # get user watchlist from db
# def get_watchlist(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))


#     watchlist_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     tconst_list = [row[1] for row in watchlist_fetch]
#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     watchlist_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst in tconst_list:
#         watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
#         x+=1

#     return watchlist_df

# # calculate % of liked attributes of film in profile
# def calculate_percentage(row):
#     total_attributes = row['total_likeable']
#     total_liked = row['num_liked_atts']
#     percentage = total_liked / total_attributes
#     return percentage 

# # count liked attributes for film
# def count_liked(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
#     cast = 0
#     if not pd.isna(row['cast']):
#         cast = len(row['cast'].split(','))
    
#     return atts + cast

# # CREATE USER PROFILE
# def get_user_profile(user_id):
#     # get user film preferences from db
#     lovedFilms = get_loved_films(user_id)
#     likedAtt = get_liked_attributes(user_id)
#     likedCast = get_liked_cast(user_id)
#     watchlist = get_watchlist(user_id)

#     # merge liked cast with liked attribute table
#     likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
#     likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
#     likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

#     # format user_profile dataframe
#     merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
#     user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
#     user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
#     user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

#     # initialise columns to calculate likeage
#     user_profile['num_liked_atts'] = 0
#     user_profile['likeage'] = 0.0
#     # count total liked attributes for each film in profile
#     for index, film in user_profile.iterrows():
#         tconst = film['tconst']
#         user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

#     # calculate likeage for each film in profile
#     user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
#     user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

#     return (user_profile, lovedFilms)

# # split user profile into likes by group
# def collate_liked_groups(user_profile):
#     group_dataframes = []

#     # Define dictionary to map group names to corresponding columns
#     group_columns = {
#         'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
#         'liked_cast': ['cast', 'likeage'],
#         'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
#         'liked_genre': ['genres', 'likeage'],
#         'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
#     }

#     # Iterate over the dictionary and create group dataframes
#     for group_name, columns in group_columns.items():
#         group_df = user_profile[columns].copy()
#         # Drop rows with all NaN values in specified columns
#         group_df = group_df.dropna(subset=columns[:-1], how='all')
#         group_dataframes.append(group_df)

#     return group_dataframes

# # cosine similarity vector with tf-idf between films in row and column
# def create_similarity_vector(row, column):
#     tfidf = TfidfVectorizer(stop_words='english')
#     row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
#     row_soup = row_soup_temp.fillna('')
#     row_matrix = tfidf.fit_transform(row_soup)

#     column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
#     column_soup = column_soup_temp.fillna('')
#     column_matrix = tfidf.transform(column_soup)

#     return linear_kernel(row_matrix, column_matrix)

# # euclidean distance vector for numerical attributes of film
# def create_euclidean_vector(row, column):
#     scaler = MinMaxScaler()
#     row = row.fillna('')
#     column = column.fillna(0)

#     row_normalized = scaler.fit_transform(row)
#     column_normalized = scaler.transform(column)
#     distances = euclidean_distances(row_normalized, column_normalized)
#     euclidean_matrix = 1 / (1 + distances)
#     return euclidean_matrix

# # get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
# def get_similar_films(vector, N):
#     filtered_vector = vector[~data['tconst'].isin(lovedFilms['tconst'])] #don't recommend film the user has already loved
#     mean_similarity = np.mean(filtered_vector, axis=1)
#     sorted_indices = np.argsort(mean_similarity)[::-1]
#     top_N = sorted_indices[:N]
#     return data.iloc[top_N]

# # function to calculate unified recommendations
# def get_unified_recommendations(user_profile_groups, similarity_vectors, N):

#     weighted_scores = {}
    
#     # scale similarity score in respective vector based on likeage of feature
#     for group, attributes in user_profile_groups.items():
#         similarity_vector = similarity_vectors[group][~data['tconst'].isin(user_profile['tconst'])] #don't recommend films already in profile
#         likeage_array = np.array(list(attributes['likeage'].tolist()))
#         weighted_similarity = similarity_vector * likeage_array

#         # meta columns have higher similarity since its numerical so adjust for this
#         if group == 'meta':
#             scaling_factor = 1 - weighted_similarity
#             weighted_similarity *= scaling_factor

#         # dictionary of groups and weighted similarity vectors 
#         weighted_scores[group] = weighted_similarity

#      # combine weighted similarity scores across all groups
#     combined_scores = np.sum(list(weighted_scores.values()), axis=0)

#     # calculate mean similarity scores
#     mean_similarity = np.mean(combined_scores, axis=1)

#     # sort the mean similarity scores and retrieve the top N indices
#     sorted_indices = np.argsort(mean_similarity)[::-1][:N]

#     return data.iloc[sorted_indices]


# data = pd.read_json('webpage/films.json')
# data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)
# data.head()


# # get update user profile and loved films
# get_profile = get_user_profile('57')
# user_profile = get_profile[0]
# lovedFilms = get_profile[1]

# # get groups of liked attributes
# grouped_likes = collate_liked_groups(user_profile)
# liked_plot = grouped_likes[0]
# liked_cast = grouped_likes[1]
# liked_crew = grouped_likes[2]
# liked_genre = grouped_likes[3]
# liked_meta = grouped_likes[4]

# # Define the list of features excluding 'likeage'
# plot_features = [col for col in liked_plot.columns if col != 'likeage']
# crew_features = [col for col in liked_crew.columns if col != 'likeage']
# cast_features = [col for col in liked_cast.columns if col != 'likeage']
# genre_features = [col for col in liked_genre.columns if col != 'likeage']
# meta_features = [col for col in liked_meta.columns if col != 'likeage']

# # Call the function with the selected columns
# plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
# crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
# cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
# genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
# meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])


# # Similarity vectors for each group
# similarity_vectors = {
#     'plot': plot_matrix,
#     'cast': cast_matrix,
#     'crew': crew_matrix,
#     'genre': genre_matrix,
#     'meta': meta_matrix
# }

# user_profile_groups = {
#     'plot': liked_plot,
#     'cast': liked_cast,
#     'crew': liked_crew,
#     'genre': liked_genre,
#     'meta': liked_meta
# }


# get_unified_recommendations(user_profile_groups, similarity_vectors, 20)



In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import mysql.connector
import warnings
warnings.filterwarnings("ignore")

data = pd.read_json('webpage/films.json')

# join and concatenate inputted columns
def create_soup(x, features):
    soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
    return ' '.join(soup_parts)

# count number of likeable elements for film
def count_likeable(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if col is not None)
    cast = len(row['cast'].split(','))
    return atts+cast

# get user loved films from db
def get_loved_films(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    rows = mycursor.fetchall()

    tconst_list = [tconst[0] for tconst in rows]

    mycursor.close()
    mydb.close()

    loved_films_df = data[data['tconst'].isin(tconst_list)]

    return loved_films_df

# get user liked attributes from db
def get_liked_attributes(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    attribute_fetch = mycursor.fetchall()

    tconst_list = [row[1] for row in attribute_fetch]
    attribute_bin = [row[1:] for row in attribute_fetch]

    mycursor.close()
    mydb.close()

    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    liked_attributes_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst, att_bin in zip(tconst_list, attribute_bin):
        row_values = [tconst]  # Initialize row values with tconst
        for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
            film_att = data[data['tconst'] == tconst]

            if liked == 1:
                row_values.append(film_att[attribute].values[0])
            else:
                row_values.append(None) 
        liked_attributes_df.loc[x] = row_values
        x+=1

    return liked_attributes_df

# get user liked cast from db
def get_liked_cast(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    cast_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    cast = [row[1:] for row in cast_fetch]
    liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

    return liked_cast_df

# get user watchlist from db
def get_watchlist(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))


    watchlist_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    tconst_list = [row[1] for row in watchlist_fetch]
    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    watchlist_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst in tconst_list:
        watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
        x+=1

    return watchlist_df

# calculate % of liked attributes of film in profile
def calculate_percentage(row):
    total_attributes = row['total_likeable']
    total_liked = row['num_liked_atts']
    percentage = total_liked / total_attributes
    return percentage 

# count liked attributes for film
def count_liked(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
    cast = 0
    if not pd.isna(row['cast']):
        cast = len(row['cast'].split(','))
    
    return atts + cast

# CREATE USER PROFILE
def get_user_profile(user_id):
    # get user film preferences from db
    lovedFilms = get_loved_films(user_id)
    likedAtt = get_liked_attributes(user_id)
    likedCast = get_liked_cast(user_id)
    watchlist = get_watchlist(user_id)

    # merge liked cast with liked attribute table
    likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
    likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
    likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

    # format user_profile dataframe
    merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
    user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
    user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
    user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

    # initialise columns to calculate likeage
    user_profile['num_liked_atts'] = 0
    user_profile['likeage'] = 0.0
    # count total liked attributes for each film in profile
    for index, film in user_profile.iterrows():
        tconst = film['tconst']
        user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

    # calculate likeage for each film in profile
    user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
    user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

    return (user_profile, lovedFilms)

# get array of names from inputted dataframe
def extract_names(data):
    names = set()
    for column in data.columns:
        for value in data[column]:
            if value and isinstance(value, str) and value.lower() not in ['none', 'null', 'nan']:
                names.update(value.split(", "))
    return list(names)

# split user profile into likes by group
def collate_liked_groups(user_profile):
    group_dataframes = []

    # Define dictionary to map group names to corresponding columns
    group_columns = {
        'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
        'liked_cast': ['cast', 'likeage'],
        'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
        'liked_genre': ['genres', 'likeage'],
        'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
    }

    # Iterate over the dictionary and create group dataframes
    for group_name, columns in group_columns.items():
        group_df = user_profile[columns].copy()
        # group_df = group_df.dropna(subset=columns[:-1], how='all')
        group_dataframes.append(group_df)

    return group_dataframes

# cosine similarity vector with tf-idf between films in row and column
def create_similarity_vector(row, column):
    tfidf = TfidfVectorizer(stop_words='english')
    row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
    row_soup = row_soup_temp.fillna('')
    row_matrix = tfidf.fit_transform(row_soup)

    column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
    column_soup = column_soup_temp.fillna('')
    column_matrix = tfidf.transform(column_soup)

    return linear_kernel(row_matrix, column_matrix)

# euclidean distance vector for numerical attributes of film
def create_euclidean_vector(row, column):
    scaler = MinMaxScaler()
    row = row.fillna('')
    column = column.fillna(0)

    row_normalized = scaler.fit_transform(row)
    column_normalized = scaler.transform(column)
    distances = euclidean_distances(row_normalized, column_normalized)
    euclidean_matrix = 1 / (1 + distances)
    return euclidean_matrix

# get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
def get_similar_films(vector, exclude):
    mean_similarity = np.mean(vector, axis=1)
    sorted_indices = np.argsort(mean_similarity)[::-1]
    top_N = sorted_indices[:]
    sorted_films = data.iloc[top_N]
    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return filtered_recommendations

# function to calculate combined recommendations
def get_combined_recommendations(user_profile_groups, similarity_vectors, exclude_films):

    weighted_scores = {}
    
    # scale similarity score in respective vector based on likeage of feature
    for group, attributes in user_profile_groups.items():
        similarity_vector = similarity_vectors[group]
        likeage_array = np.array(list(attributes['likeage'].tolist()))
        weighted_similarity = similarity_vector * likeage_array

        # dictionary of groups and weighted similarity vectors 
        weighted_scores[group] = weighted_similarity

     # combine weighted similarity scores across all groups
    combined_scores = np.sum(list(weighted_scores.values()), axis=0)

    # calculate mean similarity scores
    mean_similarity = np.mean(combined_scores, axis=1)

    # sort the mean similarity scores and retrieve the top N indices
    sorted_indices = np.argsort(mean_similarity)[::-1]

    sorted_films = data.iloc[sorted_indices]

    sorted_films['similarity'] = mean_similarity[sorted_indices] / 5

    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude_films['tconst'])]

    return filtered_recommendations



data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)


In [4]:
get_loved_films('57')

,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
16822,tt0060414,A Fine Madness,A womanizing poet falls into the hands of a ps...,5.6,"Comedy,Drama,Romance",104,1966,"Sean Connery, Patrick O'Neal, Joanne Woodward,...",Irvin Kershner,Ted D. McCord,Elliott Baker,Jerome Hellman,William H. Ziegler,John Addison,/rBLOhciaodPoG4yOulsaCjlWbWM.jpg,16
16823,tt1242777,Coopers' Camera,"Using a professional grade video camera, a dys...",6.0,Comedy,95,2008,"Boyd Banks, Mike Beaver, Jennifer Baxter, Sama...",Warren P. Sonoda,Samy Inayeh,Jason Jones,"Sean Buckley, Nicholas Tabarrok",Aden Bahadori,None,/1CLATzeeE1EF8zsBq70sYgEYH10.jpg,15
16824,tt1670916,The Box Cutter,Rob Spencer's world begins to unravel when a m...,5.8,"Drama,Mystery",100,2013,"Mark Scarboro, Brett Gentile, Irone Singleton,...",Renzo Vasquez,Carlos Pacheco,None,Michael Davis,Billy Orlando,Cutshawkane,None,15
16825,tt7335104,Goldie,"Goldie, a precocious teenager in a family shel...",5.8,Drama,88,2019,"George Sample III, Danny Hoch, Slick Woods, Ma...",Sam de Jong,Shawn Peters,Angel Gardner,"Luca Borghese, Ben Howe",None,Nathan Halpern,/qMTipS7QiGj5BLLPFUH8QyFEyBf.jpg,15
16826,tt0071607,Herbie Rides Again,The living Volkswagen Beetle helps an old lady...,5.7,"Comedy,Family,Fantasy",88,1974,"Ken Berry, John McIntire, Helen Hayes, Stefani...",Robert Stevenson,Frank V. Phillips,"Bill Walsh, Gordon Buford",None,Cotton Warburton,George Bruns,/1k14PPykkUQfhjn3xXDrc55lfHl.jpg,15
16827,tt0082547,Birgitt Haas Must Be Killed,"""Hangar"" is a new agency whose purpose is to c...",7.1,Drama,105,1981,"Philippe Noiret, Jean Rochefort, Bernard Le Co...",Laurent Heynemann,Jean-Francis Gondre,"Pierre Fabre, Caroline Huppert",Yves Gasser,None,Philippe Sarde,/pDPWQm6lVhtuSUssBhodILDGIY4.jpg,15


In [13]:
# get update user profile and loved films
get_profile = get_user_profile('57')
user_profile = get_profile[0]
lovedFilms = get_profile[1]

# get groups of liked attributes
grouped_likes = collate_liked_groups(user_profile)
liked_plot = grouped_likes[0]
liked_cast = grouped_likes[1]
liked_crew = grouped_likes[2]
liked_genre = grouped_likes[3]
liked_meta = grouped_likes[4]

# Define the list of features excluding 'likeage'
plot_features = [col for col in liked_plot.columns if col != 'likeage']
crew_features = [col for col in liked_crew.columns if col != 'likeage']
cast_features = [col for col in liked_cast.columns if col != 'likeage']
genre_features = [col for col in liked_genre.columns if col != 'likeage']
meta_features = [col for col in liked_meta.columns if col != 'likeage']

# Call the function with the selected columns
plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])

# Similarity vectors for each group
similarity_vectors = {
    'plot': plot_matrix,
    'cast': cast_matrix,
    'crew': crew_matrix,
    'genre': genre_matrix,
    'meta': meta_matrix
}

# meta_matrix
user_profile_groups = {
    'plot': liked_plot,
    'cast': liked_cast,
    'crew': liked_crew,
    'genre': liked_genre,
    'meta': liked_meta
}

# user_profile
# recommended = get_combined_recommendations(user_profile_groups, similarity_vectors, lovedFilms)
get_similar_films(genre_matrix, lovedFilms)[:20]

,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
0,tt2338355,Until the Break of Dawn,High school student Ayumi is trained by his gr...,6.4,Drama,129,2012,"Tôri Matsuzaka, Ryûta Satô, Kirin Kiki, Mirei ...",Yûichirô Hirakawa,Kôichi Nakayama,Mizuki Tsujimura,"Takuya Itô, Seiichiro Kobayashi",None,Naoki Satô,/uDq70Ghbl6k467J4UfuOwB8GQoy.jpg,15
11587,tt7870102,Critical Thinking,"Based on a true story from 1998, five Latino a...",6.5,Drama,117,2020,"John Leguizamo, Michael Kenneth Williams, Corw...",None,Zach Zamboni,Dito Montiel,"Jason Mandl, Scott M. Rosenfelt",Jamie Kirkpatrick,Chris Hajian,/l1OR2xTnOOX95KGrS3dm5YQopy5.jpg,15
20418,tt0347779,Pinjar,"In the days leading up to Partition, a Hindu w...",7.9,Drama,188,2003,"Manoj Bajpayee, Sanjay Suri, Urmila Matondkar,...",Chandra Prakash Dwivedi,Santhosh Thundiyil,Amrita Pritam,None,Ballu Saluja,Uttam Singh,/ld5kHH5HxKlgYLDlhYQpYgLymu8.jpg,15
11608,tt0184543,Birds of Exile,Tahir Agha sells all his assets when his busin...,7.4,Drama,102,1964,"Tanju Gürsu, Özden Çelik, Filiz Akin, Pervin Par",Halit Refig,Çetin Gürtop,"Orhan Kemal, Turgut Özakman",Recep Ekicigil,Mehmet Bozkus,None,/zxcZeK6n9WFsJd43cvOBizKaQyQ.jpg,15
11602,tt0055969,And Satan Calls the Turns,"On a sun-kissed beach, six people linger lazil...",5.4,Drama,82,1962,"Jacques Doniol-Valcroze, Françoise Brion, Cath...",Grisha Dabat,Raoul Coutard,Roger Vadim,Claude V. Coen,Françoise Javet,Caravelli,/NUMy13uCfDHxb6EiS82Fc8BW2d.jpg,16
20421,tt0813536,Liv,"A chamber play taking place around Christmas, ...",6.8,Drama,59,2006,"Paw Henriksen, Frederik Paarup, Malou Reymann,...",Heidi Maria Faisst,Manuel Alberto Claro,None,Meta Louise Foldager Sørensen,Jacob Thuesen,Povl Kristian,/tDgIhI8NjYpMklERObOojOfex0B.jpg,15
11599,tt0055895,Days of Wine and Roses,An alcoholic falls in love with and gets marri...,7.8,Drama,117,1962,"Jack Lemmon, Charles Bickford, Jack Klugman, L...",Blake Edwards,Philip H. Lathrop,J.P. Miller,Martin Manulis,Patrick McCormack,Henry Mancini,/cRVl1BR3x3P2NqUv61eveJNJ0ip.jpg,16
11595,tt15039894,Due Season,After being accepted into the conservatory of ...,6.1,Drama,93,2022,"Barry Shabaka Henley, Darik Bernard, Vivica A....",Cecil Chambers,Barry Markowitz,Michael Miceli,Charlie Parker-Swift,Evan Langston,Michael Mollo,/O0EucEDKlhCn9T9mNCtPQmqSws.jpg,16
11582,tt6481134,Pitta Goda,"Four good-for-nothing lads, including Tippu, f...",6.9,Drama,113,2016,"Vishwadev Rachakonda, Swaraj Rebbapragada, Pun...",Anudeep K.V.,Udhay Thangavel,None,"D. Suresh Babu, Dinesh Kumar, Ram Mohan P.",Venkata Krishna Chikkala,Kamalakar,/4cPZx2MCNCn9AjF6FCdDUIjdy32.jpg,14
20416,tt0397557,Hollow City,"A group of children, fleeing the war, is taken...",6.7,Drama,90,2004,"João Roldan, Domingos Fernandes Fonseca, Júlia...",Maria João Ganga,Jacques Besse,None,François Gonot,Pascale Chavance,"Manu Dibango, Né Gonçalves",/3bgadt5r6yuwCx65rhGWFG7tUWo.jpg,15


In [15]:
# cosine similarity vector with tf-idf between films in row and column
def create_similarity_vector(row, column):
    tfidf = TfidfVectorizer(stop_words='english')
    row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
    row_soup = row_soup_temp.fillna('')
    row_matrix = tfidf.fit_transform(row_soup)

    column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
    column_soup = column_soup_temp.fillna('')
    column_matrix = tfidf.transform(column_soup)

    return linear_kernel(row_matrix, column_matrix)

# get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
def get_similar_films(vector, exclude):
    mean_similarity = np.mean(vector, axis=1)
    sorted_indices = np.argsort(mean_similarity)[::-1]
    top_N = sorted_indices[:]
    sorted_films = data.iloc[top_N]
    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return filtered_recommendations


get_profile = get_user_profile('57')
user_profile = get_profile[0]
lovedFilms = get_profile[1]

grouped_likes = collate_liked_groups(user_profile)
liked_genre = grouped_likes[3]

genre_features = [col for col in liked_genre.columns if col != 'likeage']

genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])

genre_matrix

array([[1.        , 1.        , 0.37043822, ..., 0.55456322, 1.        ,
        0.        ],
       [1.        , 1.        , 0.37043822, ..., 0.55456322, 1.        ,
        0.        ],
       [1.        , 1.        , 0.37043822, ..., 0.55456322, 1.        ,
        0.        ],
       ...,
       [0.47214275, 0.47214275, 0.17489972, ..., 0.26183301, 0.47214275,
        0.        ],
       [0.43100091, 0.43100091, 0.15965921, ..., 0.23901725, 0.43100091,
        0.        ],
       [0.        , 0.        , 0.73065306, ..., 0.39171713, 0.        ,
        0.        ]])

In [ ]:
get_similar_films(cast_matrix, lovedFilms, 5)


In [ ]:
get_similar_films(crew_matrix, 5)


In [ ]:
get_similar_films(genre_matrix, 5)
